### 多进程  os.fork()
- Unix/Linux操作系统提供了一个os.fork()系统调用
- 调用os.fork()函数一次，则自动复制一份进程作为子进程。然后os.fork()分别在两个进程里返回。
- 注意：虽然子进程是父进程的副本。但父进程的pid和子进程的pid是不同的。
- os.fork()的返回值在父进程里是子进程pid。在子进程里是0。

In [3]:
# import os

# pid = os.fork()

# if pid == 0:
#     print('I am child process(%s) and my parent process is %s' % (os.getpid(), os.getppid()))
# else:
#     print('I am parent process(%s) and i created subprocess %s' % (os.getpid(), pid))


### 多进程 multiprocessing
- os.fork()只能用在Linux和Unix系统
- multiprocessing模块则是Linux和Unix和Windows系统都可以用
- multiprocessing模块提供一个类代表进程对象。
- 如果要启动大量进程，可以用进程池的方式创建大量进程

In [4]:
from multiprocessing import Process
import os
def run_pro(name):
    print('run process %s with pid=%s' % (name, os.getpid()))
    
if __name__ == '__main__':
    print('begin....')
    p = Process(target=run_pro, args=('ttt',))
    p.start()
    p.join()
    print('end...')

begin....
run process ttt with pid=758
end...


In [5]:
from multiprocessing import Pool
import time, os, random

def long_time_task(name):
    print('Run task %s (%s)...' % (name, os.getpid()))
    start_time = time.time()
    time.sleep(random.random() * 3)
    end_time = time.time()
    print('Task %s run %0.2f seconds' % (name, (end_time - start_time)))
    
if __name__ == "__main__":
    print('Parent process %s' % os.getpid())
    p = Pool(4)
    for i in range(5):
        p.apply_async(long_time_task, args=(i,))
    print('Waiting for all subprocesses done...')
    p.close()
    p.join()
    print('All process done')

Parent process 655
Run task 2 (958)...
Run task 1 (957)...
Run task 0 (956)...
Run task 3 (959)...
Waiting for all subprocesses done...
Task 0 run 1.01 seconds
Run task 4 (956)...
Task 1 run 1.54 seconds
Task 2 run 2.47 seconds
Task 3 run 2.84 seconds
Task 4 run 2.89 seconds
All process done


### subprocess模块可以让我们非常方便地启动一个子进程，然后控制其输入和输出。

In [6]:
import subprocess
print('$ nslookup www.python.org')
r = subprocess.call(['nslookup', 'www.python.org'])
print('Exit code:', r)

$ nslookup www.python.org
Exit code: 0


### 进程间通信
- Python的multiprocessing模块包装了底层的机制，提供了Queue、Pipes等多种方式来交换数据。

In [10]:
from multiprocessing import Process, Queue
import os, time, random

def write(q):
    print('Process to write: %s' % os.getpid())
    for value in ['a', 'b', 'c', 'd']:
        print('Put %s into queue' % value)
        q.put(value)
        time.sleep(random.random())

def read(q):
    print('Process to read: %s' % os.getpid())
    while True:
        value = q.get(True)
        print('Get %s from queue' % value)
#         time.sleep(random.random()) 这里还不能加呢，可能会在他休息时结束了。

if __name__ == '__main__':
    # 父进程创建Queue，并传给各个子进程：
    q = Queue()
    pw = Process(target=write, args=(q,))
    pr = Process(target=read, args=(q,))
    
    pw.start()
    pr.start()
    
    pw.join()
    pr.terminate() # pr进程里是死循环，无法等待其结束，只能强行终止:


Process to write: 1247
Put a into queue
Process to read: 1248
Get a from queue
Put b into queue
Get b from queue
Put c into queue
Get c from queue
Put d into queue
Get d from queue


### 多线程
- 多任务可以由多进程完成，也可以由一个进程内的多线程完成。
- 进程是由若干线程组成的，一个进程至少有一个线程。
- Python的标准库提供了两个模块。
- _thread:低级模块
- threading:高级模块，使用它就好了。

In [11]:
import threading
import time

def loop():
    print('thread %s is running' % threading.current_thread().name)
    for i in range(5):
        print('thread %s ----> %s' % (threading.current_thread().name, i))
        time.sleep(1)
    print('thread %s is ended' % threading.current_thread().name)

if __name__ == '__main__':
    print('thread %s is running' % threading.current_thread().name)
    t = threading.Thread(target=loop, name='LoopThread')
    t.start()
    t.join()
    print('thread %s is ended' % threading.current_thread().name)


thread MainThread is running
thread LoopThread is running
thread LoopThread ----> 0
thread LoopThread ----> 1
thread LoopThread ----> 2
thread LoopThread ----> 3
thread LoopThread ----> 4
thread LoopThread is ended
thread MainThread is ended


In [13]:
# 来看看多个线程同时操作一个变量怎么把内容给改乱了：
# 只要循环次数足够多，balance的结果就不一定是0了。
import threading
import time

balance = 0

def change_it(n): # 函数change_it改变balance值
    global balance
    balance = balance + n
    balance = balance - n
    
def run_thread(n): # 函数run_thread多次执行change_it函数
    for i in range(5000):
        change_it(n)

# 两个线程执行run_thread函数
t1 = threading.Thread(target=run_thread, args=(20, ))
t2 = threading.Thread(target=run_thread, args=(30, ))

t1.start()
t2.start()

t1.join()
t2.join()

print(balance)

0


In [14]:
# 正确写法如下： 
import threading
import time

balance = 0

lock = threading.Lock() # 要在主线程创建一把锁

def change_it(n): # 函数change_it改变balance值
    global balance
    balance = balance + n
    balance = balance - n
        
    
def run_thread(n): # 函数run_thread多次执行change_it函数
    for i in range(5000):
        lock.acquire() # 每次更改前，先拿到锁
        try:
            change_it(n)
        finally:
            lock.release() # 更改完释放锁。否则那些苦苦等待锁的线程将永远等待下去，成为死线程
# 两个线程执行run_thread函数
t1 = threading.Thread(target=run_thread, args=(20, ))
t2 = threading.Thread(target=run_thread, args=(30, ))

t1.start()
t2.start()

t1.join()
t2.join()

print(balance)

0


### python的全局解释器锁(GIL)
- 首先他不是python的特性， 而是Cpython解释器的特性。像Jpython等解释器就没有。
- 由于Cpython解释器的GIL的存在，导致同一时刻只能有一个线程使用CPU。
- 所以python只能使用一核CPU。也就是无法充分使用物理多核
- 因此多线程的并发在Python中就是一个美丽的梦。

### ThreadLocal模块
- 


In [15]:
import threading

# 创建全局ThreadLocal对象:
local_school = threading.local()

def process_student():
    std = local_school.student
    print('Hello, %s (in %s)' % (std, threading.current_thread().name))

def process_thread(name):
    # 绑定ThreadLocal的student:
    local_school.student = name
    process_student()

t1 = threading.Thread(target=process_thread, args=('Alice', ), name='thread_1')
t2 = threading.Thread(target=process_thread, args=('Bob', ), name='thread_2')

t1.start()
t2.start()

t1.join()
t2.join()


Hello, Alice (in thread_1)
Hello, Bob (in thread_2)


### 字符串
- 访问字符串中的值：str[index1:index2]
- 字符串更新/拼接：str + str
- 转义：\
- 字符串运算符: + -- * -- [] -- in -- not in -- r/R -- %
- 字符串格式化: 最基本的用法是将一个值插入到一个有字符串格式符 %s 的字符串中。还有format()函数
- python三引号允许一个字符串跨多行，字符串中可以包含换行符、制表符以及其他特殊字符
- 40个str函数

In [26]:
print('asddf\nklosp')

asddf
klosp


### 正则表达式 re模块
- re.match: 尝试从字符串的起始位置匹配一个模式，如果不是起始位置匹配成功的话，match()就返回none
- re.split: 用正则表达式切分字符串比用固定的字符更灵活，请看正常的切分代码：
- m.group: 如果正则表达式中定义了组，就可以在Match对象上用group()方法提取出子串来。
- m.start: 子串第一个字符的索引
- m.end: 子串最后一个字符的索引+1
- m.span: 方法返回 (start(group), end(group))。
- re.compile: 用编译后的正则表达式去匹配字符串。
- re.search 扫描整个字符串并返回第一个成功的匹配。
- re.sub: 用于替换字符串中的匹配项。
- re.findall: 在字符串中找到正则表达式所匹配的所有子串，并返回一个列表，如果没有找到匹配的，则返回空列表。
- re.finditer: 和 findall 类似，在字符串中找到正则表达式所匹配的所有子串，并把它们作为一个迭代器返回。

- 
- 一些区别
- re.match只匹配字符串的开始，如果字符串开始不符合正则表达式，则匹配失败，函数返回None；
- 而re.search匹配整个字符串，直到找到一个匹配。
- 注意： match 和 search 是匹配一次 findall 匹配所有。

In [39]:
import re
m = re.match(r'^(\d{3})\-(\d{3,8})$', '010-12345678')
print(m.group(0)) # 整串
print(m.group(1)) # 子串
print(m.group(2)) # 子串
print(m.groups()) # 子串元组

010-12345678
010
12345678
('010', '12345678')


In [29]:
re.match(r'^\d{3}-\d{3,8}$', '010 12345678')

In [30]:
# 正常切分
# 无法识别连续的空格
'a b   c '.split(' ') 

['a', 'b', '', '', 'c', '']

In [31]:
# 正则表达式切分
re.split(r'\s+', 'a b   c  d')
# 无论多少个空格都可以正常分割。加入,试试：



['a', 'b', 'c', 'd']

In [37]:
re.split(',\s+', 'a, b,   c,     d')

['a', 'b', 'c', 'd']

In [44]:
m = re.match(r'[a-z]', 'ds')
print(m.groups())

()


In [48]:
line = "Cats are smarter than dogs"
m = re.match(r'(.*) are (.*?) (.*)', line, re.M|re.I)
print(m.group())
print(m.group(0))
print(m.group(1))
print(m.group(2))
print(m.group(3))
print(m.groups())
print(m.start(1))
print(m.end(1))
print(m.span(1))

Cats are smarter than dogs
Cats are smarter than dogs
Cats
smarter
than dogs
('Cats', 'smarter', 'than dogs')
0
4
(0, 4)


In [51]:
m = re.match(r'www', 'www baidu com and www goole com')
print(m.group())
print(m.span())
# match()起始位置匹配，则成功返回

www
(0, 3)


In [54]:
# m = re.match(r'www', 'was baidu com and www goole com')
# print(m.group())
# print(m.span())
# 起始位置不匹配，则返回None

In [52]:
m = re.search(r'www', 'was baidu com and www goole com')
print(m.group())
print(m.span())
# 搜索匹配，找到第一个即停止

www
(18, 21)


In [56]:
m = re.findall(r'www', 'was baidu com and www goole www')
print(m)
# 搜索匹配，找到全部即停止，返回结果是列表

['www', 'www']


In [59]:
m = re.finditer(r'www', 'was baidu com and www goole www')
print(m)
# 搜索匹配，找到全部即停止，返回结果是迭代器，但元素是匹配对象，不是直接值
for i in m:
    print(i.group())

www
www
